In [1]:
import sys
import os

# Get the directory of the current notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, ".."))

from modeling.rating import rating_get_df

rating_df = rating_get_df()

rating_df

,side,role,rating,season,week,team
0,def,qb1,-160.166667,2024,3,WSH
1,def,rb1,-36.166667,2024,3,WSH
2,def,rb2,-32.333333,2024,3,WSH
3,def,wr1,-59.000000,2024,3,WSH
4,def,wr2,-53.666667,2024,3,WSH
...,...,...,...,...,...,...
58003,off,rb1,41.161458,2024,4,ARI
58004,off,rb2,28.200000,2024,4,ARI
58005,off,wr1,44.642857,2024,4,ARI
58006,off,wr2,65.980952,2024,4,ARI


In [2]:
import pandas as pd

from modeling.compare import (
    compare_teams_rating,
)

from modeling.schedule import schedule_next_week

next_week = schedule_next_week()

results = pd.DataFrame(
    [
        compare_teams_rating(
            next_week["season"], next_week["week"]-1, game["home"], game["away"]
        )
        for game in next_week["games"]
    ]
)

results.sort_values(by=["diff"])

Exception: Can not find: WAS > WAS

In [11]:
import pandas as pd
from stats import stats_roles, get_role_week


def _compare_teamss(team1_df, team2_df):
    off_df = team1_df[team1_df["side"] == "off"].reset_index().set_index("role")
    def_df = team2_df[team2_df["side"] == "def"].reset_index().set_index("role")

    return {
        role: off_df.loc[role]["rating"] - def_df.loc[role]["rating"]
        for role in stats_roles
    }


alias = {
    "NYG": "NYG",
    "DAL": "DAL",
    "NO": "NO",
    "ATL": "ATL",
    "LAR": "STL",
    "CHI": "CHI",
    "MIN": "MIN",
    "GB": "GB",
    "PIT": "PIT",
    "IND": "IND",
    "DEN": "DEN",
    "NYJ": "NYJ",
    "PHI": "PHI",
    "TB": "TB",
    "CIN": "CIN",
    "CAR": "CAR",
    "JAX": "JAX",
    "HOU": "HOU",
    "WSH": "WSH",
    "ARI": "ARI",
    "NE": "NE",
    "SF": "SF",
    "CLE": "CLE",
    "LV": "OAK",
    "KC": "KC",
    "LAC": "SD",
    "BUF": "BUF",
    "BAL": "BAL",
    "TEN": "TEN",
    "MIA": "MIA",
    "SEA": "SEA",
    "DET": "DET",
}


def compare_teamss(season, week, team1, team2, show_vs=False):
    rating_df = get_rating_df()

    team1_alias = alias[team1] if team1 in alias else team1
    team2_alias = alias[team2] if team2 in alias else team2

    week_df = rating_df[(rating_df["season"] == season) & (rating_df["week"] == week)]

    team1_df = week_df[week_df["team"] == team1_alias]
    if len(team1_df.index) == 0:
        print("==========")
        print(rating_df[(rating_df["season"] == season)])
        print("----------")
        print(week_df["team"])
        raise Exception(f"Can not find: {team1} > {team1_alias}")
    team2_df = week_df[week_df["team"] == team2_alias]
    if len(team2_df.index) == 0:
        raise Exception(f"Can not find: {team2} > {team2_alias}")

    team1_res = _compare_teamss(team1_df, team2_df)
    team1_res["rating"] = sum(team1_res.values())
    team1_res["team"] = team1
    if show_vs:
        team1_res["vs"] = team2

    team2_res = _compare_teamss(team2_df, team1_df)
    team2_res["rating"] = sum(team2_res.values())
    team2_res["team"] = team2
    if show_vs:
        team2_res["vs"] = team1

    return pd.DataFrame([team1_res, team2_res])


schedule = [
    {"home": "NYG", "away": "DAL"},
    {"home": "NO", "away": "ATL"},
    {"home": "LAR", "away": "CHI"},
    {"home": "MIN", "away": "GB"},
    {"home": "PIT", "away": "IND"},
    {"home": "DEN", "away": "NYJ"},
    {"home": "PHI", "away": "TB"},
    {"home": "CIN", "away": "CAR"},
    {"home": "JAX", "away": "HOU"},
    {"home": "WSH", "away": "ARI"},
    {"home": "NE", "away": "SF"},
    {"home": "CLE", "away": "LV"},
    {"home": "KC", "away": "LAC"},
    {"home": "BUF", "away": "BAL"},
    {"home": "TEN", "away": "MIA"},
    {"home": "SEA", "away": "DET"},
]


def compare_week(season, week):
    return pd.concat(
        [compare_teamss(season, week, game["home"], game["away"]) for game in schedule]
    )


comparisions = compare_week(2024, 3)


def sort_and_return(df, role):
    rtn = df.sort_values(by=[role], ascending=False).head(3)

    rtn.style.format(precision=2)

    return rtn.to_markdown()


print("--qb1--")
print(sort_and_return(comparisions, "qb1"))

print("--rb1--")
print(sort_and_return(comparisions, "rb1"))

print("--rb2--")
print(sort_and_return(comparisions, "rb2"))

print("--wr1--")
print(sort_and_return(comparisions, "wr1"))

print("--wr2--")
print(sort_and_return(comparisions, "wr2"))

print("--wr3--")
print(sort_and_return(comparisions, "wr3"))

--qb1--
|    |      wr1 |     wr2 |     wr3 |     rb1 |      rb2 |     qb1 |   rating | team   |
|---:|---------:|--------:|--------:|--------:|---------:|--------:|---------:|:-------|
|  0 |  85.7447 | 82.2868 | 65.8441 | 77.8833 |  23.8728 | 228.779 |  564.41  | PIT    |
|  0 |  84.9585 | 85.6637 | 30.5622 | 27.9117 | -17.9709 | 214.847 |  425.973 | BUF    |
|  1 | 131.432  | 23.7611 | 56.6405 | 52.0055 |  47.4079 | 193.135 |  504.382 | ARI    |
--rb1--
|    |       wr1 |     wr2 |      wr3 |     rb1 |      rb2 |        qb1 |   rating | team   |
|---:|----------:|--------:|---------:|--------:|---------:|-----------:|---------:|:-------|
|  1 | -22.5542  | 98.8507 | -6.24731 | 85.6684 | -90.9032 |   6.65143  |  71.4658 | BAL    |
|  0 |  85.7447  | 82.2868 | 65.8441  | 77.8833 |  23.8728 | 228.779    | 564.41   | PIT    |
|  0 |   6.50887 | 49.8581 |  6.45084 | 77.1385 | -15.797  |  -0.143714 | 124.016  | NYG    |
--rb2--
|    |      wr1 |      wr2 |     wr3 |      rb1 |     rb2 |  

In [13]:
from stats import get_game_df


def run_week(season, week):
    games_df = get_game_df()

    week_df = games_df[(games_df["season"] == season) & (games_df["week"] == week)]

    for i, row in week_df[
        ["homeTeamDisplay", "homeScore", "awayTeamDisplay", "awayScore"]
    ].iterrows():
        print(
            f"{row['homeTeamDisplay']} ({row['homeScore']}) v {row['awayTeamDisplay']} ({row['awayScore']})"
        )
        print(
            compare_teamss(
                season, week, row["homeTeamDisplay"], row["awayTeamDisplay"]
            ).to_markdown()
        )


run_week(2023, 16)

PHI (33) v NYG (25)
|    |     wr1 |     wr2 |     wr3 |     rb1 |      rb2 |      qb1 |   rating | team   |
|---:|--------:|--------:|--------:|--------:|---------:|---------:|---------:|:-------|
|  0 | 64.241  | 39.2184 | 70.2475 | 46.6664 | 46.2403  |  77.9902 |  344.604 | PHI    |
|  1 | 44.3977 | 48.5702 | 46.2201 | 22.7463 |  5.21033 | 104.347  |  271.492 | NYG    |
KC (14) v OAK (20)
|    |      wr1 |     wr2 |     wr3 |     rb1 |     rb2 |      qb1 |   rating | team   |
|---:|---------:|--------:|--------:|--------:|--------:|---------:|---------:|:-------|
|  0 | 45.7199  | 54.328  | 63.3834 | 57.5385 | 37.8086 | 103.228  |  362.006 | KC     |
|  1 |  3.93499 | 60.9445 | 36.3043 | 48.4163 | 62.5798 |  35.2914 |  247.471 | OAK    |
STL (30) v NO (22)
|    |     wr1 |     wr2 |     wr3 |     rb1 |      rb2 |      qb1 |   rating | team   |
|---:|--------:|--------:|--------:|--------:|---------:|---------:|---------:|:-------|
|  0 | 59.3201 | 64.1056 | 55.0507 | 33.842  | 22.64

In [7]:
get_role_week(2024, 3, "ATL", "wr2")

season                              2024
week                                   3
gameId                         401671793
gameDisplay                    KC vs ATL
gameDate               2024-09-23T00:20Z
teamId                                 1
teamDisplay                          ATL
playerId                         4040655
playerDisplay             Darnell Mooney
playerPosition                        wr
playerPositionGroup             catching
playerPositionNorm                    wr
passCmp                                0
passAtt                                0
passYds                                0
passTd                                 0
passInt                                0
passLong                               0
passRating                             0
passTargetYds                          0
rushAtt                                0
rushYds                                0
rushTd                                 0
rushLong                               0
rushYdsBc       

In [14]:
from stats import get_all_games

results = {"tp": 0, "fp": 0, "tn": 0, "fn": 0}

try:
    for index, row in (
        get_game_df().sort_values(by=["season", "week"], ascending=False).iterrows()
    ):
        print(
            "analyzing > ",
            row["season"],
            row["week"],
            row["homeTeamDisplay"],
            row["awayTeamDisplay"],
        )
        compare = compare_teamss(
            row["season"], row["week"], row["homeTeamDisplay"], row["awayTeamDisplay"]
        )
        home_stats = compare.iloc[0]
        away_stats = compare.iloc[1]

        home_expected = home_stats["rating"] > away_stats["rating"]
        home_won = row["homeScore"] > row["awayScore"]

        print(
            f"results >> {home_stats['rating']} > {away_stats['rating']} >> {row['homeScore']} > {row['awayScore']}"
        )
        if home_won:
            if home_expected:
                results["tp"] += 1
            else:
                results["fn"] += 1
        else:
            if home_expected:
                results["fp"] += 1
            else:
                results["tn"] += 1

except:
    print("out of data")

print("results: ", results)

analyzing >  2024 3 CIN WSH
results >> 531.2254314718798 > 251.2592682077402 >> 33 > 38
analyzing >  2024 3 PIT SD
results >> 336.3081767694671 > 135.55883307572412 >> 20 > 10
analyzing >  2024 3 MIN HOU
results >> 375.6599196502422 > 264.3444576559852 >> 34 > 7
analyzing >  2024 3 OAK CAR
results >> 209.7061554085629 > 509.63231281431626 >> 22 > 36
analyzing >  2024 3 SEA MIA
results >> 149.74803504022248 > 200.0761037400806 >> 24 > 3
analyzing >  2024 3 BUF JAX
results >> 515.5263518286573 > 66.53439541291495 >> 47 > 10
analyzing >  2024 3 NO PHI
results >> 440.2060909209665 > 250.15701484895035 >> 12 > 15
analyzing >  2024 3 TB DEN
results >> 221.19845828181784 > 176.42056154506201 >> 7 > 26
analyzing >  2024 3 DAL BAL
results >> 385.36672251817834 > 145.72926808854226 >> 25 > 28
analyzing >  2024 3 ARI DET
results >> 174.85489781359797 > 53.897694114633566 >> 13 > 20
analyzing >  2024 3 STL SF
results >> 311.28912411978524 > 467.3255593518308 >> 27 > 24
analyzing >  2024 3 ATL KC
r